In [1]:
import torch
import torch.nn as nn

In [3]:
class selfAttension(nn.Module):
    def __init__(self, embed_size, heads):
        super(selfAttension, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)  
        self.query = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.value = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc = nn.Linear(heads*self.head_dim, embed_size, bias=False)
        self.softmax = nn.Softmax(-1)

    def forward(self, keys, query, value, mask):
        N= query.shape[0]
        keys_len, query_len, value_len = keys.shape[1], query.shape[1], value.shape[1]

        #split keys and query along the batch_size, sequence_length, heads, and head_dim
        keys = keys.reshape(N, keys_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)
        value = value.reshape(N, value_len, self.heads, self.head_dim)

        keys = self.keys(keys)
        query = self.query(query)
        value = self.value(value)

        #calculate the score
        score =  torch.einsum("nqhd,nkhd->nhqk", [query, keys]) 
        # query shape N(n), query_len(q), heads(h), head_dim(d)
        # keys shape N(n), keys_len(k), heads(h), head_dim(d)
        # score shape N(n), heads(h), query_len(q), keys_len(k)
        # so we need to transpose the score to N, query_len, heads, keys_len
        # so nqhd * nkhd = nhqk

        if mask is not None:
            score = score.masked_fill(mask == 0, float("-1e20"))

        score = self.softmax(score / (self.embed_size ** (1/2)))

        #i need value*score 
        # value shape N(n), value_len(v), heads(h), head_dim(d)
        # score shape N(n), heads(h), query_len(q), keys_len(k)
        # the output shape of feature vector should be N(n), query_len(q), heads(h), head_dim(d)
        #so use einsum to calculate the output
        out = torch.einsum("nhqv,nvhd->nqhd", [score, value]).reshape(N, query_len, self.heads*self.head_dim)

        out = self.fc(out)  
        return out


In [4]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion): 
        super(TransformerBlock, self).__init__()
        self.attension = selfAttension(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion*embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion*embed_size, embed_size)
        )
        self.norm2 = nn.LayerNorm(embed_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, keys, query, value, mask):
        attention = self.attension(keys, query, value, mask)    
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out
        

class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embed_size, num_layers, heads, dropout, device, forward_expansion, max_length):
        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device

        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.positional_embedding = nn.Embedding(max_length, embed_size)
        self.layers = nn.ModuleList()

        for _ in range(num_layers):
            self.layers.append(TransformerBlock(embed_size, heads, dropout = dropout, forward_expansion = forward_expansion))

        self.dropout = nn.Dropout(dropout)    

    def forward(self, x, mask):
        N, seq_len = x.shape 
        positions = torch.arange(0, seq_len).expand(N, seq_len).to(self.device)

        out = self.dropout((self.word_embedding(x)+self.positional_embedding(positions)))
        for layer in self.layers:
            out = layer(out, out, out,mask)

        return out 

In [5]:
class DecoderBlock(nn.Module):
    def __init__ (self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.device = device
        self.attension = selfAttension(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        
        self.transformer_block = TransformerBlock(embed_size, heads, dropout, forward_expansion)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x,key, value, src_mask, tgt_mask):
        attention = self.attension(x, x, x, tgt_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(key, query, value, src_mask)
        return out
    

class Decoder(nn.Module):
    def __init__ ( self, tgt_vocab_size, embed_size, num_layers, heads, dropout, device, forward_expansion, max_length):
        super(Decoder, self).__init__()
        self.device = device

        self.word_embedding = nn.Embedding (tgt_vocab_size, embed_size)
        self.positional_embedding = nn.Embedding(max_length, embed_size)
        self.layers = nn.ModuleList()

        for _ in range(num_layers):
            self.layers.append( DecoderBlock(embed_size, heads, forward_expansion, dropout, device))

        self.fc_out = nn.Linear(embed_size, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)    

    def forward(self, x, enc_out, src_mask, tgt_mask):
        N, seq_len = x.shape
        positions = torch.arange(0, seq_len).expand(N, seq_len).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.positional_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, tgt_mask)

        out = self.fc_out(x)
        return out

In [6]:
class Transformer(nn.Module):
    def __init__(
            self, 
            src_vocab_size, 
            tgt_vocab_size, 
            src_pad_idx,
            tgt_pad_idx,
            num_layers = 6, 
            heads= 8, 
            embed_size = 512, 
            device = "cuda", 
            forward_expansion = 4, 
            dropout = 0.10,
            max_length = 100
        ):
        super(Transformer, self).__init__()
        self.encoder = Encoder(src_vocab_size, embed_size, num_layers, heads, dropout, device, forward_expansion, max_length)
        self.decoder = Decoder(tgt_vocab_size, embed_size, num_layers, heads, dropout, device, forward_expansion, max_length)
        self.src_pad_idx = src_pad_idx
        self.tgt_pad_idx = tgt_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)    
    
    def make_tgt_mask(self, tgt):
        N, tgt_len = tgt.shape
        tgt_mask = torch.tril(torch.ones(tgt_len, tgt_len)).expand(N, 1, tgt_len, tgt_len) 
        return tgt_mask.to(self.device)

    def forward(self, src, tgt):
        src_mask = self.make_src_mask(src)
        tgt_mask = self.make_tgt_mask(tgt)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(tgt, enc_src, src_mask, tgt_mask)
        return out  
    


In [58]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(
        device
    )
    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

    src_pad_idx = 0
    trg_pad_idx = 0
    src_vocab_size = 10
    trg_vocab_size = 10
    model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
        device
    )
    out = model(x, trg[:, :-1])
    print(out.shape)

cuda
torch.Size([2, 7, 10])
